In [60]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [61]:
%sendtofile --source UDPblackviewphone.py
%sendtofile --source ubxminimal.py

Sent 65 lines (1965 bytes) to ubxminimal.py.


In [62]:
%sendtofile main.py

import time, machine, urandom

# delay long enough to interrupt if we need to, 
# before the uart or wlan bricks it for debugging
pled = machine.Pin(2, machine.Pin.OUT)
for i in range(11):
    pled.value(i%2)
    time.sleep_ms(500)  

from ubxminimal import uartUBX, initUBX
initUBX()

from UDPblackviewphone import dwrite, dwriteUBX, connectBlackview

for i in range(11):
    pled.value(i%2)
    time.sleep_ms(200)

connectBlackview(pled)

for i in range(11):
    pled.value(i%2)
    time.sleep_ms(100)

while True:
    x = uartUBX.read()
    if x:
        dwriteUBX(x)
        print(x[:5])
    else:
        time.sleep_ms(5)
    pled.value(1-pled.value())



Sent 33 lines (659 bytes) to main.py.


In [40]:
%disconnect

attempt to exit paste mode
[\r\x03\x02] b'\r\nMicroPython v1.9.4-782-geb446ec22 on 2019-01-22; ESP module with ESP8266\r\nType "help()" for more information.\r\n>>> '
Closing serial Serial<id=0x7faa019dd208, open=True>(port='/dev/ttyUSB2', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)
